In [1]:
""" Alpaca Paper Trade with the 1st LSTM """

# import sys
# import lib
# sys.modules["src.lib"] = lib

import os

if "src" not in os.listdir():
    os.chdir("../../../")

import json

import time
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone
import pickle
import warnings

import pandas_market_calendars as mcal
from keras.models import load_model

from src.lib.alpaca_historical import AlpacaData
from src.lib.alpaca_paper import AlpacaTrader
from src.lib.activations import negative_softmax
from src.lib.price_history import transform_price_history, PriceHistory
from src.lib.stock_dataset import StockDataset

In [2]:
path = "results/1_lstm_ptrade/"
keys = "alpaca_config.json"
isPaper = True

In [6]:
def now_utc():
    return datetime.now().astimezone(tz=timezone("UTC"))


def get_next_trading_minute():
    nyse = mcal.get_calendar("NYSE")
    now = now_utc()
    lookahead = timedelta(days=0)
    while True:
        lookahead += timedelta(hours=1)

        schedule = nyse.schedule(start_date=now, end_date=now + lookahead)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
            valid_minutes = mcal.date_range(schedule, frequency="1T")            
        
        where_future = np.where(valid_minutes > now)[0]
        if len(where_future) > 0:
            break

    return valid_minutes[where_future[0]]
    # return (now + timedelta(minutes=1)).replace(second=0)

def get_last_n_minutes(n_valid):
    nyse = mcal.get_calendar("NYSE")
    now = now_utc()
    lookbehind = timedelta(days=0)
    n_iter = 0
    while True:
        lookbehind += timedelta(minutes=n_valid * 2**n_iter)
        n_iter += 1

        schedule = nyse.schedule(start_date=now - lookbehind, end_date=now)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
            valid_minutes = mcal.date_range(schedule, frequency="1T")            
        
        where_past = np.where(valid_minutes < now)[0]
        if len(where_past) > n_valid:
            break

    return valid_minutes[where_past[-n_valid:]]


In [4]:
api_data = AlpacaData(keys)
api_trade = AlpacaTrader(keys, isPaper)
model = load_model(
    os.path.join(path, "model.h5"),
    custom_objects={
        "negative_softmax": negative_softmax
    }
)

with open(os.path.join(path, "config.json"), "r") as f:
    config = json.load(f)

n_time_steps = config["n_time_steps"]
symbols = config["symbols"]
target_column = config["target_column"]

with open(os.path.join(path, "scaler.pkl"), "rb") as f:
    scaler = pickle.load(f)

ta_preprocess_candles = 296

In [10]:
def trade_loop():

    valid_mins = get_last_n_minutes(ta_preprocess_candles + 1)
    data_start_time = valid_mins[0]

    # Sleep until next trading time
    next_trade_time = get_next_trading_minute()
    sleep_seconds = (next_trade_time - now_utc()).total_seconds() + 1
    print("Sleeping for", timedelta(seconds=sleep_seconds))
    time.sleep(sleep_seconds)


    # Get the last 64 candles
    # data_start_time = next_trade_time - timedelta(minutes=n_time_steps + 1 + ta_preprocess_candles) 
    price_histories = []
    for symbol in symbols:
        history = api_data.get_bars(symbol, data_start_time, next_trade_time, log=False)
        if len(history) == 0:
            print(f"Warning: No data between {data_start_time} and {next_trade_time}")
            return
        price_history = PriceHistory(history, symbol)
        # TODO accept **kwargs here
        price_history = transform_price_history(price_history)
        price_histories.append(price_history)
    
    # Process the candes
    dataset = StockDataset(price_histories, target_column, n_time_steps)
    dataset.apply_standard_scaler(scaler)
    X = dataset.prediction_X(n_time_steps)
    
    # Model makes predictions
    preds = model.predict(X)[0]
    next_holdings = dict(zip(symbols, preds))
    print("Desired holdings", next_holdings)
    responses = api_trade.update_holdings(next_holdings, verbose=True)

    # What prices did we get when we bought/sold for this step of trading

    with open(os.path.join(path, "trade_log.txt"), "a") as f:
        f.write("%s,%s,%s\n" % (str(now_utc()), str(next_holdings), str(responses)))

while True:
    trade_loop()

Sleeping for 0:00:10.915381
DATA LEN 296
DATA LEN 296
Desired holdings {'SPY': -0.42962942, 'SPXS': -0.48474693}
https://data.alpaca.markets/v2/stocks/SPY/snapshot
sell SPY 12
https://data.alpaca.markets/v2/stocks/SPXS/snapshot
buy SPXS 590
[<Response [200]>, <Response [200]>]
